# **DID IT?**

**No. It never rains in Seattle anymore. Not since Supersonics left the city.**

<img src="https://media.giphy.com/media/l46C9VsWTWD9CHocg/giphy.gif">

All jokes a side. Supersonics were amazing by the way. In this notebook, you will see a quick cleaning and reshaping to data. Then a little visualization and finally predictions. Enjoy!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/did-it-rain-in-seattle-19482017/seattleWeather_1948-2017.csv')

In [ ]:
print(df.shape)
df.head()

In [ ]:
df['RAIN'].value_counts()

Checking for *null* values.

In [ ]:
for column in df:
    print(column)
    print(df[column].isnull().sum().sum())

There are very few *null* values so we can drop them.

In [ ]:
df.dropna(inplace=True)

I will make *RAIN* column numeric before I start with predictions but for analysis it is nicer to see the label names.

I do thing every column is important for this classification problem. Because there are few of them. But it would be nice to see their effect visually.

In [ ]:
sns.pairplot(df, hue='RAIN')

In [ ]:
sns.displot(data=df, x='TMIN', y='TMAX', hue='RAIN', kind='kde')

It seems an easy classification problem when you see it. But uncertainty of *not raining* is tricky as you can see on the above.

Now we need to make RAIN column numeric.

In [ ]:
rain = pd.get_dummies(df['RAIN'], drop_first=True)

In [ ]:
df = df.drop('RAIN', axis=1)
df = pd.concat([df, rain], axis=1)
print(df.shape)
df.head()

In [ ]:
df.columns = ['DATE', 'PRCP', 'TMAX', 'TMIN', 'RAIN']
df.head()

We are good to go.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
x = df[['PRCP', 'TMAX', 'TMIN']]
y = df['RAIN']

In [ ]:
x.shape, y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

**LOGISTIC REGRESSION**

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(x_train, y_train)
prediction_lr = logistic.predict(x_test)
print(classification_report(y_test,prediction_lr))

**DECISION TREE**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)
prediction_dt = tree.predict(x_test)
print(classification_report(y_test, prediction_dt))

**RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(x_train, y_train)
prediction_rf = forest.predict(x_test)
print(classification_report(y_test, prediction_rf))

**XGBOOST**

In [ ]:
import xgboost
xgb = xgboost.XGBClassifier()
xgb.fit(x_train,y_train)
prediction_xgb = xgb.predict(x_test)
print(classification_report(y_test, prediction_xgb))

Traditional models performed great. This dataset is just perfect for traditional machine learning techniques: big dataset with few number of variables. Finally I want to try the data with a artificial neural network.

**ANN**

In [ ]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout

In [ ]:
x.shape

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_dim=3),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size=10, epochs=10,verbose=2)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['accuracy'], label='Accuracy', color='blue')
plt.plot(history.history['loss'], label='Loss', color='red')
plt.legend()

In [ ]:
prediction_nn = model.predict(x_test)
prediction_nn = [1 if y>=0.5 else 0 for y in prediction_nn]
print(classification_report(y_test, prediction_nn))

This is it for this notebook. Take care.

<img src="https://media.giphy.com/media/xUPOqo6E1XvWXwlCyQ/giphy.gif">